In [ ]:
import math
def strength_calc(x):
    st = math.log2(x+1)+(x/10)
    return st

In [ ]:
import random
def utility_calc(maxV, minV):
    i = random.randint(0,1)
    rndm = random.randint(1, 10)
    util = strength_calc(maxV) - strength_calc(minV) + ((-1)**i)*(rndm/10)
    return util

In [ ]:
def alpha_beta_pruning_minmax(maxplyr, alpha, beta, depth, n_idx, vals):
    if depth == 0:
        vals = vals[n_idx]
        return vals
    if maxplyr:
        max_eval = float('-inf')
        for idx in range(2):
            evaluate = alpha_beta_pruning_minmax(False, alpha, beta, depth - 1, n_idx * 2 + idx, vals)
            max_eval = max(max_eval, evaluate)
            alpha = max(alpha, evaluate)
            if beta <= alpha:
                break
        return max_eval

    else:
        min_eval = float('inf')
        for indx in range(2):
            evaluate = alpha_beta_pruning_minmax(True, alpha, beta, depth - 1, n_idx * 2 + indx, vals)
            min_eval = min(min_eval, evaluate)
            beta = min(beta, evaluate)
            if beta <= alpha:
                break
        return min_eval

In [ ]:
def det_winner(score, max_player, win_counts):
    if score > 0:
        winner = f'{max_player} (Max)'
        win_counts[max_player] += 1
    elif score < 0:
        if max_player == "Magnus Carlsen":
            winner = "Fabiano Caruana (Min)"
            win_counts["Fabiano Caruana"] += 1
        else:
            winner = "Magnus Carlsen (Min)"
            win_counts["Magnus Carlsen"] += 1
    else:
        winner = 'Draw'
        win_counts['Draw'] += 1
    return winner

In [ ]:
def score_calc_simulator(maxplyr, maxV, minV):
    val = []
    for i in range(2**5):
        util = utility_calc(maxV, minV)
        val.append(util)
    score = alpha_beta_pruning_minmax(maxplyr, -float('inf'), float('inf'), 5, 0, val)
    return score

In [ ]:
def chess(starting_player, carlsen_strength, caruana_strength):
    results = []
    win_counts = {'Magnus Carlsen': 0, 'Fabiano Caruana': 0, 'Draw': 0}
    for i in range(4):
        if starting_player == 0:
            maxV, minV = carlsen_strength, caruana_strength
            max_player = 'Magnus Carlsen'
            max_plyr = True
        else:
            maxV, minV = caruana_strength, carlsen_strength
            max_player = 'Fabiano Caruana'
            max_plyr = False
        score = score_calc_simulator(max_plyr, maxV, minV)
        winner = det_winner(score, max_player, win_counts)

        results.append((winner, score))
        print(f'Game {i + 1} Winner: {winner} (Utility value: {score:.2f})')

        # Swap starting player for next game
        starting_player = 1 - starting_player


    print("\n")
    print('Overall Results:')
    print(f'Magnus Carlsen Wins: {win_counts["Magnus Carlsen"]}')
    print(f'Fabiano Caruana Wins: {win_counts["Fabiano Caruana"]}')
    print(f'Draws: {win_counts["Draw"]}')

    if win_counts["Magnus Carlsen"] > win_counts["Fabiano Caruana"]:
        print('Overall Winner: Magnus Carlsen')
    elif win_counts["Fabiano Caruana"] > win_counts["Magnus Carlsen"]:
        print('Overall Winner: Fabiano Caruana')
    else:
        print('Overall Winner: Draw')

# Input handling
starting_player = int(input('Enter starting player for game 1 (0 for Carlsen, 1 for Caruana): '))
carlsen_strength = float(input('Enter base strength for Carlsen: '))
caruana_strength = float(input('Enter base strength for Caruana: '))

# Start the games
chess(starting_player, carlsen_strength, caruana_strength)


Enter starting player for game 1 (0 for Carlsen, 1 for Caruana): 0
Enter base strength for Carlsen: 9
Enter base strength for Caruana: 8
Game 1 Winner: Fabiano Caruana (Min) (Utility value: -0.15)
Game 2 Winner: Magnus Carlsen (Min) (Utility value: -0.55)
Game 3 Winner: Magnus Carlsen (Max) (Utility value: 0.35)
Game 4 Winner: Magnus Carlsen (Min) (Utility value: -0.15)


Overall Results:
Magnus Carlsen Wins: 3
Fabiano Caruana Wins: 1
Draws: 0
Overall Winner: Magnus Carlsen


# **Part 2**

In [ ]:
def maxim(maxplyr, alpha, beta, depth, n_idx, vals):
    if depth == 0:
        vals = vals[n_idx]
        return vals
    if maxplyr:
        max_eval = float('-inf')
        for idx in range(2):
            evaluate = maxim(True, alpha, beta, depth - 1, n_idx * 2 + idx, vals)
            max_eval = max(max_eval, evaluate)
            alpha = max(alpha, evaluate)
        return max_eval

In [ ]:
def mind_control(strt_plr, cost, light_strn, l_strn):
    leaf_vals = []
    if strt_plr == 0:
        maxV, minV = (light_strn, l_strn)
    else:
        maxV, minV = (l_strn, light_strn)
    for i in range(2**5):
        util = utility_calc(maxV, minV)
        leaf_vals.append(util)
    normal_score = alpha_beta_pruning_minmax(True, -math.inf, math.inf, 5, 0, leaf_vals)
    mind_control_score = maxim(True, -math.inf, math.inf, 5, 0, leaf_vals)
    mind_control_final = mind_control_score - cost

    if strt_plr == 0:
        player_name = "Light"
    else:
        player_name = "L"
    print("\n")
    print(f'Minimax value without Mind Control: {normal_score:.2f}')
    print(f'Minimax value with Mind Control: {mind_control_score:.2f}')
    print(f'Minimax value with Mind Control after incurring the cost: {mind_control_final:.2f}')

    if normal_score > 0 and mind_control_final > 0:
        print(f"{player_name} should NOT use Mind Control as the position is already winning.")
    elif mind_control_final > normal_score and mind_control_final > 0:
        print(f"{player_name} should use Mind Control.")
    elif mind_control_final < 0 and normal_score < 0:
        print(f"{player_name} should NOT use Mind Control as the position is losing either way.")
    elif mind_control_final<0 and normal_score>0:
        print(f"{player_name} should NOT use Mind Control as it backfires.")


# Input values for problem 2
strt_plyr = int(input("Enter who goes first (0 for Light, 1 for L): "))
mind_control_cost = float(input("Enter the cost of using Mind Control: "))
light_strn = float(input("Enter base strength for Light: "))
l_strn = float(input("Enter base strength for L: "))

mind_control(strt_plyr, mind_control_cost, light_strn, l_strn)


Enter who goes first (0 for Light, 1 for L): 0
Enter the cost of using Mind Control: 0.25
Enter base strength for Light: 10
Enter base strength for L: 8


Minimax value without Mind Control: 0.69
Minimax value with Mind Control: 0.99
Minimax value with Mind Control after incurring the cost: 0.74
Light should NOT use Mind Control as the position is already winning.


In [ ]:
# import random
# import math

# # Strength function
# def strength(x):
#     return math.log2(x + 1) + x / 10

# # Utility function
# def utility(maxV, minV):
#     i = random.choice([0, 1])
#     rand_component = (-1) ** i * random.randint(1, 10) / 10
#     return strength(maxV) - strength(minV) + rand_component

# # Minimax with Alpha-Beta Pruning
# def alphabeta(depth, max_depth, alpha, beta, is_maximizing, maxV, minV):
#     if depth == max_depth:
#         return utility(maxV, minV)

#     if is_maximizing:
#         value = float('-inf')
#         for _ in range(2):  # Two possible moves
#             value = max(value, alphabeta(depth + 1, max_depth, alpha, beta, False, maxV, minV))
#             alpha = max(alpha, value)
#             if beta <= alpha:
#                 break
#         return value
#     else:
#         value = float('inf')
#         for _ in range(2):
#             value = min(value, alphabeta(depth + 1, max_depth, alpha, beta, True, maxV, minV))
#             beta = min(beta, value)
#             if beta <= alpha:
#                 break
#         return value

# # Simulate Problem 1
# def simulate_games(starting_player, carlsen_strength, caruana_strength):
#     carlsen_wins = 0
#     caruana_wins = 0
#     draws = 0

#     for game in range(4):
#         is_carlsen_max = (game % 2 == 0 and starting_player == 0) or (game % 2 == 1 and starting_player == 1)
#         maxV = carlsen_strength if is_carlsen_max else caruana_strength
#         minV = caruana_strength if is_carlsen_max else carlsen_strength

#         value = alphabeta(0, 5, float('-inf'), float('inf'), True, maxV, minV)
#         value = round(value, 2)

#         winner = ""
#         if value > 0:
#             winner = "Magnus Carlsen (Max)" if is_carlsen_max else "Fabiano Caruana (Max)"
#             if "Carlsen" in winner:
#                 carlsen_wins += 1
#             else:
#                 caruana_wins += 1
#         elif value < 0:
#             winner = "Fabiano Caruana (Min)" if is_carlsen_max else "Magnus Carlsen (Min)"
#             if "Carlsen" in winner:
#                 carlsen_wins += 1
#             else:
#                 caruana_wins += 1
#         else:
#             winner = "Draw"
#             draws += 1

#         print(f"Game {game + 1} Winner: {winner} (Utility value: {value})")

#     print("\nOverall Results:")
#     print(f"Magnus Carlsen Wins: {carlsen_wins}")
#     print(f"Fabiano Caruana Wins: {caruana_wins}")
#     print(f"Draws: {draws}")
#     if carlsen_wins > caruana_wins:
#         print("Overall Winner: Magnus Carlsen")
#     elif caruana_wins > carlsen_wins:
#         print("Overall Winner: Fabiano Caruana")
#     else:
#         print("Overall Winner: Draw")


# simulate_games(0, 9, 8)



Game 1 Winner: Magnus Carlsen (Max) (Utility value: 0.55)
Game 2 Winner: Magnus Carlsen (Min) (Utility value: -0.05)
Game 3 Winner: Magnus Carlsen (Max) (Utility value: 0.75)
Game 4 Winner: Fabiano Caruana (Max) (Utility value: 0.15)

Overall Results:
Magnus Carlsen Wins: 3
Fabiano Caruana Wins: 1
Draws: 0
Overall Winner: Magnus Carlsen
